## Introducción al Machine Learning 
### Regresión Lineal, Correlación Serial y Generación de Características

Este ejemplo muestra como una regresión lineal simple muestra el problema de 
una correlación serial, y como se puede corregir introdiciendo una nueva característca

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Prima Seguro
El problema consiste en reconstruir la función que utiliza una 
entidad para asignar la prima de seguro de vida para una hipoteca
dado que al cliente solo le muestran a modo de ejemplo una tabla con la prima a 
pagar sobre un monto particular. 
El fichero de datos contiene para cada edad la prima a pagar por cada 100K de hipoteca

In [ ]:
prima_seguro = pd.read_csv('../data/prima_seguro.csv')

In [ ]:
plt.scatter(prima_seguro.edad, prima_seguro.prima)

In [ ]:
prima_seguro.head()

Haremos la estimación de los parámetros de la regresión lineal
a partir de la fórmula y con la librería sklearn

$$ \beta = \frac{cov(X,Y)}{var(X)} $$

$$ \alpha = \bar{Y} - \beta\bar{X}$$

Numpy calcula una matriz de covarianzas para un conjunto de variables

In [ ]:
np.cov(prima_seguro.edad, prima_seguro.prima)

In [ ]:
cov_prima = np.cov(prima_seguro.edad, prima_seguro.prima)[1][0]
cov_prima

In [ ]:
var_edad = np.var(prima_seguro.edad)
var_edad

In [ ]:
beta = cov_prima/var_edad
beta

In [ ]:
alpha = np.average(prima_seguro.prima) - beta*np.average(prima_seguro.edad)
alpha

Hacemos la gráfica para verificar como se ajusta la recta
a la muestra de puntos. Para la recta necesitamos dos puntos (x,y)

In [ ]:
X = np.array([18, 75])
Y = alpha + beta*X
print(X)
print(Y)

In [ ]:
plt.plot(X,Y, 'r')
plt.scatter(prima_seguro.edad, prima_seguro.prima)
plt.grid(True)

Hecemos la misma regresión pero con la regresión lineal de sklearn

In [ ]:
from sklearn.linear_model import LinearRegression

Como tenemos un vector con la variable independiente, necesitamos llevarla 
a una matriz (N,1) haciendo el reshape 

In [ ]:
x_train = prima_seguro.edad.values.reshape(-1,1)
y_train = prima_seguro.prima

In [ ]:
LM = LinearRegression()
reg = LM.fit(x_train, y_train)

In [ ]:
reg.score(x_train, y_train)

Los parámetros de la regresión: coef_ corresponde a $\beta$ e intercet_ al $\alpha$

In [ ]:
print(reg.coef_, reg.intercept_)

In [ ]:
X2 = np.array([18, 75])
Y2 = reg.intercept_ + reg.coef_*X

In [ ]:
plt.plot(X,Y, 'r')
plt.plot(X2,Y2, 'g')
plt.grid()
plt.scatter(prima_seguro.edad, prima_seguro.prima)

Graficamos la distancia de los residuos para confirmar
una correlación serial como consecuencia de tener un modelo
mal especificado

In [ ]:
prima_hat = alpha + beta*prima_seguro.edad

In [ ]:
fig, ax = plt.subplots()
ax.plot(X,Y, 'r')
ax.scatter(prima_seguro.edad,prima_seguro.prima)
residual = prima_seguro.prima - prima_hat
ax.vlines(prima_seguro.edad,prima_seguro.prima,prima_hat)
plt.grid()
plt.show()

Como la nube de puntos tiene forma de parábola buscamos
especificar el modelo con la fórmula correspondiente.

$$y = w_0 + w_1x + w_2x^2$$


**OJO**: El modelo tiene que ser lineal en los parámetros, no en las variables.  A efectos del modelo es como tener una característica nueva $$x_2 = edad\times edad$$

In [ ]:
x_train_extra = np.concatenate([x_train, (x_train**2)], axis=1)

In [ ]:
reg2 = LM.fit(x_train_extra, y_train)

In [ ]:
print(reg2.coef_, reg2.intercept_)

In [ ]:
y2_predict = reg2.predict(x_train_extra)

In [ ]:
reg2.score(x_train_extra, y_train)

In [ ]:
fig, ax = plt.subplots()

ax.scatter(prima_seguro.edad,prima_seguro.prima)
ax.plot(prima_seguro.edad, y2_predict, 'r')

#residual = prima_seguro.prima - y2_predict
ax.vlines(prima_seguro.edad,prima_seguro.prima,y2_predict)
plt.grid()
plt.show()

Esta gráfica está mejor ajustada, con la correlación serial prácticamente eliminada